Running this notebooks prepares everything for the course

## Load global settings

In [2]:
import json
import yaml


with open("settings.yaml") as f:
    settings = yaml.safe_load(f)


# Global canvas settings
COURSE_ID = settings["global"]["canvas"]["course_id"]

# Paths
RESOURCES_PATH = settings["global"]["paths"]["resources"]
SUBMISSIONS_PATH = settings["global"]["paths"]["submissions"]

# Weeks
WEEKS = settings["weeks"]

# Randomization
GROUPS = settings["global"]["randomization"]["groups"]
SEED = settings["global"]["randomization"]["seed"]

# Surveys
SURVEY_QUESTIONS = settings["surveys"]["survey_questions"]
SURVEY_DEFINITIONS = settings["surveys"]["survey_definitions"]

## Jsonifying resources

Assignments, rubrics, example solutions, and learning goals are broken up according to the question indicators and converted into Json files for each week.

These Json files are later used to prompt the LLM questionwise.

In [3]:
# Package imports
from scripts.jsonify import jsonify_resources, analyze_jsonify_results

In [ ]:
# Jsonify
results = {f"week-{week}": jsonify_resources(week, RESOURCES_PATH) for week in WEEKS}
analyze_jsonify_results(results)

## Prepare submissions directory

The submissions directory is prepared before the course

In [5]:
# Package imports
from canvasapi import Canvas
from canvasapi.requester import Requester
from canvas_connector.utils.canvas_utils import get_students_df
import os

# Local imports
from scripts.filehandling import make_student_folders

# Get environment variables
CANVAS_API_URL = os.getenv("CANVAS_API_URL")
CANVAS_API_KEY = os.getenv("CANVAS_API_KEY")

# Prepare connection to Canvas API
canvas = Canvas(CANVAS_API_URL, CANVAS_API_KEY)
requester = Requester(CANVAS_API_URL, CANVAS_API_KEY)

In [ ]:
# Get course from canvas
course = canvas.get_course(COURSE_ID)

# Get all students enrolled in the course
students_df = get_students_df(course)
students_df = students_df.loc[students_df["user_name"] != "Luke Korthals"] # Remove Luke

# Create student folders
for student_id in students_df["user_id"]:
    [make_student_folders(student_id, week, SUBMISSIONS_PATH) for week in WEEKS]
    
print(f"folders for {len(students_df)} students created")

## Randomize students into groups
- Students are randomly assigned to one of 6 groups
    - ABAB
    - BABA
    - ABBA
    - BAAB
    - AABB
    - BBAA
- Each group has an associated section in the Canvas course which were manually created in the webinterface
- Students are assigned to sections based on the group they are in

In [7]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt

In [9]:
run_randomization = True # this was already run

In [ ]:
if run_randomization:
    # Set seed
    np.random.seed(SEED)

    # Create balanced list of groups
    n = len(students_df)
    n_groups = len(GROUPS)
    sample_from = np.tile(GROUPS, n // n_groups)
    if len(sample_from) < n:
        sample_from = np.append(sample_from, np.random.choice(GROUPS, n - len(sample_from), replace=False))

    # Check if groups are balanced
    plt.hist(sample_from, bins=n_groups)

    # shuffle groups and assign to students
    assignment = np.random.shuffle(sample_from)
    students_df["group"] = sample_from
    for i, week in enumerate(WEEKS):
        students_df[f"week-{week}"] = students_df["group"].apply(lambda x: x[i])

    # Save student groups
    students_df.to_csv("submissions/students.csv", index=False)
    students_df.head()

    # # Assign students to Canvas sections
    # sections = [section for section in course.get_sections()]
    # for student_id, group in zip(students_df["user_id"], students_df["group"]):
    #     for section in sections:
    #         if group == section.name:
    #             section.enroll_user(user=student_id)


In [11]:
# add example student to all groups
import pandas as pd
students_df = pd.read_csv("submissions/students.csv")
canvas_students = get_students_df(course)
canvas_students = canvas_students.loc[canvas_students["user_name"] == "Luke Korthals"]
canvas_students["group"] = "AAAA"
canvas_students["week-1"] = "A"
canvas_students["week-2"] = "A"
canvas_students["week-3"] = "A"
canvas_students["week-4"] = "A"
students_df = pd.concat([students_df, canvas_students])
students_df.to_csv("submissions/students.csv", index=False)


## Preparing the surveys

- All surveys that are defined in the `settings.yaml` are created in the Canvas course.
- Afterwards, they are manually assigned to the associated sections

In [ ]:
create_surveys = False # this was already run

In [27]:
# Local imports
from scripts.survey_utils import create_survey

In [29]:
if create_surveys:
    # Create all surveys
    for key in SURVEY_DEFINITIONS:
        create_survey(course, SURVEY_DEFINITIONS[key], SURVEY_QUESTIONS)